# COMP30830 - Software Engineering
# Project: DublinBikes
Group work by:
- **Avik Saha** (18200379)
- **Rafael Martins** (18200630)
- **Giovanni Facchinetti** (18202941)

# Machine Learning prediction model
In this notebook we develop a ML model to predict two key values that are functional to our web application:
- **number of available bikes** for a selected station
- **number of available bike stands** for a selected station

The model is trained on data fetched from an Amazon RDS database, where we store historical data referring to:
- real-time information that Dublin Bikes makes available through their web API
- Dublin weather information from OpenWeather, obtained by a web API

**N.B.**: all database data are kept up-to-date.

## Getting started
First of all, we need to import Python packages that are required to our model, as well as fetching the data from the Amazon RDS database.
### Import required packages
A number of Python packages are required for our model in order to work, so we import them in our Notebook.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import os
import pymysql
import pandas as pd
import sys

/Users/giovannifacchinetti/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Connect to Amazon RDS database
After having stored all the necessary credentials in specific variables, we connect to the database providing error-handling in case of connection issues.

In [2]:
# Declare and initialize database connection credentials
host = 'dublinbikes.cb2pu3bkmmlf.us-east-2.rds.amazonaws.com'
user = 'master'
password = 'master-50'
db = 'dublinbikes'

# Attempt connection to database
# Print a statement on the screen to check whether the connection is working
try:
    con = pymysql.connect(host = host, user = user, password = password, db = db, use_unicode = True, charset = 'utf8')
    print('+=========================+')
    print('|  CONNECTED TO DATABASE  |')
    print('+=========================+')
    
# Exit if connection not working   
except Exception as e:
        sys.exit(e)

+=========================+
|  CONNECTED TO DATABASE  |
+=========================+


### Data fetching

#### Bikes information
We store real-time data from Dublin Bikes in a relation named **"availability"** in the RDS database schema.

We fetch Pandas dataframe object and we examine it.

**N.B.**:We started scrapping the bikes data on 22/02/2019

In [3]:
# Create dataframe and store data running SQL query
df = pd.read_sql_query("SELECT * FROM dublinbikes.availability", con)

# Examine dataframe object, show first 10 rows
df.head(10)

,ID,number,available_bikes,available_bike_stands,last_update,day_of_week,hour_d,mins,date_of_fetching
0,1,42,12,18,2147483647,Friday,22,13,2019-02-22
1,2,30,3,17,2147483647,Friday,22,13,2019-02-22
2,3,54,4,29,2147483647,Friday,22,13,2019-02-22
3,4,108,11,28,2147483647,Friday,22,13,2019-02-22
4,5,56,2,38,2147483647,Friday,22,13,2019-02-22
5,6,6,5,14,2147483647,Friday,22,13,2019-02-22
6,7,18,14,16,2147483647,Friday,22,13,2019-02-22
7,8,32,24,6,2147483647,Friday,22,13,2019-02-22
8,9,52,0,32,2147483647,Friday,22,13,2019-02-22
9,10,48,3,37,2147483647,Friday,22,13,2019-02-22


In [4]:
# Show number of rows and columns
print("The dataset has %s rows and %s columns." % df.shape)

The dataset has 648733 rows and 9 columns.


Since 113 stations provide real-time information, we show the first 3 entries in our dataframe (first 113*3 = 339 rows)

In [5]:
# Show first 3 real-time bikes entries
df.head(339)

,ID,number,available_bikes,available_bike_stands,last_update,day_of_week,hour_d,mins,date_of_fetching
0,1,42,12,18,2147483647,Friday,22,13,2019-02-22
1,2,30,3,17,2147483647,Friday,22,13,2019-02-22
2,3,54,4,29,2147483647,Friday,22,13,2019-02-22
3,4,108,11,28,2147483647,Friday,22,13,2019-02-22
4,5,56,2,38,2147483647,Friday,22,13,2019-02-22
5,6,6,5,14,2147483647,Friday,22,13,2019-02-22
6,7,18,14,16,2147483647,Friday,22,13,2019-02-22
7,8,32,24,6,2147483647,Friday,22,13,2019-02-22
8,9,52,0,32,2147483647,Friday,22,13,2019-02-22
9,10,48,3,37,2147483647,Friday,22,13,2019-02-22


#### Weather information
We store weather data from OpenWeather in a relation named **"weather"** in the RDS database schema.

We fetch Pandas dataframe object and we examine it.

**N.B.**:We started scrapping the weather data on 22/02/2019

In [6]:
# Create dataframe and store data running SQL query
df_2 = pd.read_sql_query("SELECT * FROM dublinbikes.weather", con)

# Examine dataframe object, show first 50 rows
df_2.head(50)

,ID,temperature,day_of_week,wind_speed,cloud_coverage,hour_d,mins,date_of_fetching
0,1,12.58,Friday,4.6,Clouds,22,13,2019-02-22
1,2,11.51,Friday,7.7,Clouds,22,28,2019-02-22
2,3,11.51,Friday,7.7,Clouds,22,43,2019-02-22
3,4,11.51,Friday,7.7,Clouds,22,58,2019-02-22
4,5,11.58,Friday,7.2,Clouds,23,13,2019-02-22
5,6,11.59,Friday,7.2,Clouds,23,28,2019-02-22
6,7,11.51,Friday,7.7,Clouds,23,43,2019-02-22
7,8,11.57,Friday,7.7,Clouds,23,58,2019-02-22
8,9,11.57,Saturday,7.7,Clouds,0,13,2019-02-23
9,10,11.58,Saturday,7.7,Clouds,0,28,2019-02-23


In [7]:
# Show number of rows and columns
print("The dataset has %s rows and %s columns." % df_2.shape)

The dataset has 5519 rows and 8 columns.


## Data preparation
### Unique values functional to merging
We need to **merge the bikes and weather dataframes**.<br/>
Since we have stored in the database a **scraping-timestamp** for each single row in all our relations, we determine the **unique minutes** of fetching (we assume the hours are fully represented 0-24 as data are fetched roughly every 15 minutes).

In [10]:
print("Unique values for:\n- mins in weather:", pd.unique(df_2.mins.ravel()))

Unique values for:
- mins in weather: [13 28 43 58 14 29 44 59 30 45  0 15 31 46  1 16 32 47  2 17  3 18 33 48
 19 34 49  4  5 20 35 50 21 36 51  6  7 22 37 52 23 38 53  8 24 39 54  9
 10 25 40 55 11 26 41 56 57 12 27 42]


In [11]:
print("Unique values for:\n- mins in availability:", pd.unique(df.mins.ravel()))

Unique values for:
- mins in availability: [13 21 28 35 42 49 56  3 10 18 25 32 39 46 53  0  8 15 22 29 36 43 50 57
  5 12 19 26 33 40 48 55  2  9 16 23 30 38 45 52 59  6 20 17  7 47 37 27
  4 11  1 51 58 41 24 31 14 54 44 34]


### Merge dataframes
Now that we have unique values in common between the two dataframes in terms of scraping timestamps (we want the data to be consistent), we **proceed to merge them using a SQL inner join** and store the result in a new dataframe object.

Note the reduction in terms of tuples (rows) from the original bikes dataframe.

In [12]:
# Merge dataframes
mergedStuff = pd.merge(df, df_2, on = ['mins', 'date_of_fetching', 'hour_d'], how = 'inner')
# Show first 2 scraps (oldest) for stations (113*2=226)
mergedStuff.head(226)

,ID_x,number,available_bikes,available_bike_stands,last_update,day_of_week_x,hour_d,mins,date_of_fetching,ID_y,temperature,day_of_week_y,wind_speed,cloud_coverage
0,1,42,12,18,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
1,2,30,3,17,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
2,3,54,4,29,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
3,4,108,11,28,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
4,5,56,2,38,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
5,6,6,5,14,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
6,7,18,14,16,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
7,8,32,24,6,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
8,9,52,0,32,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds
9,10,48,3,37,2147483647,Friday,22,13,2019-02-22,1,12.58,Friday,4.6,Clouds


In [13]:
# Show last 2 scraps (most recent) for stations (113*2=226)
mergedStuff.tail(226)

,ID_x,number,available_bikes,available_bike_stands,last_update,day_of_week_x,hour_d,mins,date_of_fetching,ID_y,temperature,day_of_week_y,wind_speed,cloud_coverage
49268,647717,42,26,4,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49269,647718,30,0,20,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49270,647719,54,28,5,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49271,647720,108,29,11,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49272,647721,56,4,36,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49273,647722,6,11,9,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49274,647723,18,16,14,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49275,647724,32,4,26,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49276,647725,52,26,6,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds
49277,647726,48,9,31,2147483647,Sunday,20,40,2019-04-14,5511,6.96,Sunday,6.2,Clouds


In [14]:
# Show number of rows and columns of the new joined dataframe
print("The dataset has %s rows and %s columns." % mergedStuff.shape)

The dataset has 49494 rows and 14 columns.


### Dummy coding
We need to incorporate the following information (variables) in our model, but they are not immediately usable as they come in a categorical form:
- **cloud coverage**
- **day of the week**

Thus, we transform them in a **series of dummy variables**. The process is known as "dummy coding".
#### Weather: cloud coverage
To incorporate cloud coverage information in our ML model, we need to identify the unique values representing **cloud coverage** information and create dummy variables for each.

In [16]:
print("Unique values for:\n- cloud coverage in weather:", pd.unique(df_2.cloud_coverage.ravel()))

Unique values for:
- cloud coverage in weather: ['Clouds' 'Clear' 'Rain' 'Fog' 'Drizzle' 'Mist' 'Snow']


#### Timestamp: days of the week
We assume that the unique values for week days are known, we can proceed to dummy coding.

In [17]:
# Create a separate dataframe with days of the week (categorical)
data_input1 = pd.DataFrame(mergedStuff['day_of_week_x'])

# Create a separate dataframe with cloud coverage information (categorical)
data_input2 = pd.DataFrame(mergedStuff['cloud_coverage'])

# Concatenate the two dataframes in the main one
dummy = pd.get_dummies(data_input1)
dummy_2 = pd.get_dummies(data_input2)
mergedStuff = pd.concat([mergedStuff,dummy],axis=1)
mergedStuff = pd.concat([mergedStuff,dummy_2],axis=1)

In [20]:
# Show last two station scraps and check dummy coded columns
mergedStuff.tail(226)

,ID_x,number,available_bikes,available_bike_stands,last_update,day_of_week_x,hour_d,mins,date_of_fetching,ID_y,...,day_of_week_x_Sunday,day_of_week_x_Thursday,day_of_week_x_Tuesday,day_of_week_x_Wednesday,cloud_coverage_Clear,cloud_coverage_Clouds,cloud_coverage_Drizzle,cloud_coverage_Fog,cloud_coverage_Mist,cloud_coverage_Rain
49268,647717,42,26,4,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49269,647718,30,0,20,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49270,647719,54,28,5,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49271,647720,108,29,11,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49272,647721,56,4,36,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49273,647722,6,11,9,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49274,647723,18,16,14,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49275,647724,32,4,26,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49276,647725,52,26,6,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0
49277,647726,48,9,31,2147483647,Sunday,20,40,2019-04-14,5511,...,1,0,0,0,0,1,0,0,0,0


### Check dataframe state
#### New columns
The new dummy coded variables have been concatenated at the end of the dataframe, so we now check our dataframe shape.

In [21]:
# Show number of rows and columns of the dataframe
print("The dataset has %s rows and %s columns." % mergedStuff.shape)

The dataset has 49494 rows and 27 columns.


#### Data types
As a further check, we analyze the data type in our dataframe.

In [23]:
mergedStuff.dtypes

ID_x                         int64
number                       int64
available_bikes              int64
available_bike_stands        int64
last_update                  int64
day_of_week_x               object
hour_d                       int64
mins                         int64
date_of_fetching            object
ID_y                         int64
temperature                float64
day_of_week_y               object
wind_speed                 float64
cloud_coverage              object
day_of_week_x_Friday         uint8
day_of_week_x_Monday         uint8
day_of_week_x_Saturday       uint8
day_of_week_x_Sunday         uint8
day_of_week_x_Thursday       uint8
day_of_week_x_Tuesday        uint8
day_of_week_x_Wednesday      uint8
cloud_coverage_Clear         uint8
cloud_coverage_Clouds        uint8
cloud_coverage_Drizzle       uint8
cloud_coverage_Fog           uint8
cloud_coverage_Mist          uint8
cloud_coverage_Rain          uint8
dtype: object

## ML model
In order to predict our two target variables
- **number of available bikes**
- **number of available bike stands**
we need two differnt ML models that are trained in the following section.
After having tested the regression model as a viable alternative, we decided to implement a Random Forest classifier model, as it proves to be a more effective predictor.

**N.B.**: we train our models on a random selection of 2/3 of the original dataset. We perform testing on the remaining 1/3. 

### Predict the number of available bikes

#### Model training

In [26]:
# Select model features and store them in a new dataframe
input_model = pd.DataFrame(mergedStuff[['number', 'hour_d', 'mins', 'temperature', 'wind_speed', 'day_of_week_x_Friday', 'day_of_week_x_Monday', 'day_of_week_x_Saturday', 'day_of_week_x_Sunday', 'day_of_week_x_Thursday', 'day_of_week_x_Tuesday', 'day_of_week_x_Wednesday', 'cloud_coverage_Clear', 'cloud_coverage_Clouds', 'cloud_coverage_Drizzle', 'cloud_coverage_Fog', 'cloud_coverage_Mist', 'cloud_coverage_Rain']])

# Define target variable
output = mergedStuff['available_bikes']

# Split dataset to train and test
X_train,X_test,Y_train,Y_test=train_test_split(input_model,output,test_size=0.33,random_state=42)
print("Training the model on %s rows and %s columns." % X_train.shape)

# Instantiate RandomForestRegressor object calling 100 decision tree models
rf = RandomForestRegressor(n_estimators=100)

# Train the model
rf.fit(X_train, Y_train)

print("Testing the model on %s rows." % Y_test.shape[0])

Training the model on 33160 rows and 18 columns.
Testing the model on 16334 rows.


In [ ]:
#LinearReg=LinearRegression()
#LinearReg.fit(X_train,Y_train)

In [27]:
#pred=LinearReg.predict(X_test)
#pred1=pd.DataFrame(pred)
#print (pred1)
#print(X_test)
#print(np.array(X_test))

       number  hour_d  mins  temperature  wind_speed  day_of_week_x_Friday  \
43622      56       9    43         7.60         7.7                     0   
26241      17      17    42         8.13         1.5                     0   
35910     100      18    31         9.22         3.6                     0   
47119      21      15    48         9.08         5.1                     1   
11478      25       6    22         5.54         4.6                     0   
17179     108       8    37         8.41         9.3                     1   
26554      88      22    42         4.68         1.0                     0   
6927       67       1    48         4.32         1.5                     0   
30006      76       1    22         4.21         2.1                     0   
16028      33       2    40         4.27         6.2                     0   
24385      24      22    39         9.64         5.1                     0   
13360      45      17     8         1.46         8.7            

In [28]:
# Show test data
print(X_test)

       number  hour_d  mins  temperature  wind_speed  day_of_week_x_Friday  \
43622      56       9    43         7.60         7.7                     0   
26241      17      17    42         8.13         1.5                     0   
35910     100      18    31         9.22         3.6                     0   
47119      21      15    48         9.08         5.1                     1   
11478      25       6    22         5.54         4.6                     0   
17179     108       8    37         8.41         9.3                     1   
26554      88      22    42         4.68         1.0                     0   
6927       67       1    48         4.32         1.5                     0   
30006      76       1    22         4.21         2.1                     0   
16028      33       2    40         4.27         6.2                     0   
24385      24      22    39         9.64         5.1                     0   
13360      45      17     8         1.46         8.7            

#### Model testing

In [29]:
# Get prediction for test cases
prediction = rf.predict(X_test)

# Show the predicted test cases
print(prediction)
print(Y_test)

[31.94  7.43 16.2  ... 24.01 14.68  9.  ]
43622    40
26241     6
35910    16
47119    15
11478     6
17179     0
26554     3
6927     36
30006    21
16028     1
24385    15
13360    23
14737     0
29298     6
11605    28
12815    18
26150    24
48114    16
13920     9
8530      0
10444     5
9672      1
4962      4
38133    18
23314     0
28661     0
24396     7
33555    22
35431    13
5395      0
         ..
27619    12
4227     22
35244    20
47616     3
37186     0
3406     23
25836    12
46030     0
41015     5
5557      4
25902     5
9592      2
48229     2
41994     1
6597      0
15825     1
23417    15
12249     7
12558    21
5992      0
9238      0
4430     38
38574     2
28397    18
11370    19
8451     24
3165      3
8105     28
30235    14
21690     0
Name: available_bikes, Length: 16334, dtype: int64


#### Model evaluation
In order to evaluate the prediction effectiveness of our model, we compute the **root-mean-square deviation**, defined as the standard deviation of the residuals (prediction errors).

In [34]:
print("RMSE: %f" % np.sqrt(metrics.mean_squared_error(Y_test,prediction)))

RMSE: 5.760331


#### Module integration
In order to connect the ML model to our Flask web application, we need to produce a 'prediction-data' file from the trained model using the **Pickle** Python module.<br/>
Pickle allows us to store the prediction model in a file that we save on the server, in order to be used by the application to actually deliver a prediction based on the requested stations by the user.

In [ ]:
import pickle

pickle.dump(rf, open('final_prediction.pickle', 'wb'))

In [ ]:
# This is not strictly functional to the application
random_forest = pickle.load(open("final_prediction.pickle", "rb"))

### Predict the number of available bike stands

#### Model training

In [42]:
# Select model features and store them in a new dataframe
input_model_available_bike_stands = pd.DataFrame(mergedStuff[['number', 'hour_d', 'mins', 'temperature', 'wind_speed', 'day_of_week_x_Friday', 'day_of_week_x_Monday', 'day_of_week_x_Saturday', 'day_of_week_x_Sunday', 'day_of_week_x_Thursday', 'day_of_week_x_Tuesday', 'day_of_week_x_Wednesday', 'cloud_coverage_Clear', 'cloud_coverage_Clouds', 'cloud_coverage_Drizzle', 'cloud_coverage_Fog', 'cloud_coverage_Mist', 'cloud_coverage_Rain']])

# Define target variable
output_bike_stands=mergedStuff['available_bike_stands']

# Split dataset to train and test
X_train_2, X_test_2, Y_train_2, Y_test_2 = train_test_split(input_model_available_bike_stands, output_bike_stands, test_size = 0.33, random_state = 42)

print("Training the model on %s rows and %s columns." % X_train_2.shape)

# Instantiate RandomForestRegressor object calling 100 decision tree models
rf_2 = RandomForestRegressor(n_estimators = 100)

# Train the model
rf_2.fit(X_train_2, Y_train_2)

print("Testing the model on %s rows." % Y_test_2.shape[0])

Training the model on 33160 rows and 18 columns.
Testing the model on 16334 rows.


#### Model testing

In [43]:
# Get prediction for test cases
prediction_bike_stands = rf_2.predict(X_test_2)

# Show the predicted test cases
print (prediction_bike_stands)
print(Y_test_2)

[ 7.58 13.34  7.2  ... 16.96 24.27 20.73]
43622     0
26241    14
35910     9
47119    15
11478    24
17179    40
26554    27
6927      4
30006    17
16028    22
24385     5
13360     7
14737    40
29298    23
11605    12
12815    12
26150    16
48114     9
13920    29
8530     40
10444    35
9672     29
4962     17
38133    22
23314    40
28661    40
24396    13
33555     8
35431    27
5395     40
         ..
27619    28
4227      8
35244     0
47616    37
37186    40
3406      7
25836    28
46030    25
41015    35
5557     26
25902    15
9592     18
48229    38
41994    35
6597     40
15825    19
23417    15
12249    22
12558     9
5992     40
9238     16
4430      2
38574    28
28397    22
11370    21
8451      1
3165     17
8105     12
30235    26
21690    30
Name: available_bike_stands, Length: 16334, dtype: int64


#### Model evaluation
In order to evaluate the prediction effectiveness of our model, we compute the **root-mean-square deviation**, defined as the standard deviation of the residuals (prediction errors).

In [39]:
print("RMSE: %f" % np.sqrt(metrics.mean_squared_error(Y_test_2,prediction_bike_stands)))

RMSE: 5.794453


#### Module integration
In order to connect the ML model to our Flask web application, we need to produce a 'prediction-data' file from the trained model using the **Pickle** Python module.<br/>
Pickle allows us to store the prediction model in a file that we save on the server, in order to be used by the application to actually deliver a prediction based on the requested stations by the user.

In [40]:
import pickle

pickle.dump(rf_2,open('final_prediction_bike_stands.pickle', 'wb'))

In [41]:
# This is not strictly functional to the application
random_forest_stands=pickle.load(open("final_prediction_bike_stands.pickle", "rb"))